In [1]:
import torch
import torchvision.transforms as transforms
from LookGenerator.datasets.tps_dataset import ShirtsDataset
from torch.utils.data import DataLoader
from LookGenerator.networks.bpgm.model.models import BPGM
from LookGenerator.networks.bpgm.train_bpgm import train_bpgm
import torch.nn.functional as F
import numpy as np
import os
import PIL.Image as Image
import matplotlib.pyplot as plt
from LookGenerator.networks_training.utils import check_path_and_creat
from LookGenerator.networks.utils import save_model, load_model

In [2]:
transform = transforms.Compose([
    transforms.Resize((256,192)),
   transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])
mask_transform = transforms.Compose([
    transforms.Resize((256,192)),
])


In [3]:
batch_size = 1
num_workers = 0
pin_memory = True

In [4]:
dataset = ShirtsDataset(
    root=r'C:\Users\Даша\кто\мусор\zalando-hd-resized\train',
    #r"C:\Users\DenisovDmitrii\Desktop\zalando-hd-resize\train",
    transform=transform,
    transform_mask=mask_transform
)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=pin_memory, num_workers=num_workers)

In [5]:
save_directory = r'C:\Users\DenisovDmitrii\OneDrive - ITMO UNIVERSITY\peopleDetector\tps\weights\test'
check_path_and_creat(save_directory)

RuntimeError: 

In [5]:
for data in dataloader:
    break

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [7]:
model = BPGM(device=device)

initialization method [normal]
initialization method [normal]


In [8]:
loss = train_bpgm(dataloader=dataloader,
                  model=model, device=device, epochs=1)

C:\Users\Даша\PycharmProjects\SMBackEnd\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Даша\PycharmProjects\SMBackEnd\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0%|          | 6/11647 [00:53<29:04:29,  8.99s/it]


KeyboardInterrupt: 

In [ ]:
save_model(model.to('cpu'), path=f"{save_directory}\\epoch_02.pt")

# тест

In [21]:
testtransform = transforms.Compose([
    transforms.Resize((256,192)),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])
testmask_transform = transforms.Compose([
    transforms.Resize((256,192)),
   ])


In [22]:
test_dataset = ShirtsDataset(
    root=r"C:\Users\Даша\кто\мусор\zalando-hd-resized\test",
    transform=testtransform,
    transform_mask=testmask_transform
)

In [23]:
topil = transforms.ToPILImage()
toten = transforms.ToTensor()

In [24]:
testdataloader = DataLoader(test_dataset, batch_size=1, pin_memory=True)

In [25]:
weights_dir = r'C:\Users\Даша\кто\мусор\epoch_02.pt'
#r"C:\Users\DenisovDmitrii\OneDrive - ITMO UNIVERSITY\peopleDetector\tps\weights\test\epoch_00.pt"

In [26]:
model = load_model(model, weights_dir)

In [10]:
count = 0
for data in testdataloader:
    mask = data['segmentation'].to(device)
    shirt = data['shirt'].to(device)
    shirt_mask = data['shirt_mask'].to(device)

    model = model.to(device)
    theta = model(mask, shirt)

    warped = F.grid_sample(shirt, theta, padding_mode='border', align_corners=True).to('cpu')

    warped_mask = F.grid_sample(shirt_mask, theta, padding_mode='border', align_corners=True).to('cpu')
    warped_mask = topil(warped_mask[0])
    warped_mask.show()
    warped = topil(warped[0]/2+0.5)
    warped.show()
    count += 1
    if count == 4:
        break

подготовка

In [28]:
path_people = r'C:\Users\Даша\кто\мусор\zalando-hd-resized\test\imageWithNoCloth'
#r'C:\Users\DenisovDmitrii\Desktop\forEncoderNew\train\imageWithNoCloth'

In [29]:
cloud_people = os.listdir(path_people)

In [30]:
len(cloud_people)

2032

In [32]:
# ИСПОЛЬЗОВАТЬ batch_size=1
count = 0
for index, data in enumerate(testdataloader):

    cloud_person = Image.open(path_people + "\\" + cloud_people[index])
    cloud_person = toten(cloud_person)

    mask = data['segmentation'].to(device)
    shirt = data['shirt'].to(device)
    cwm = data['torso'].to(device)

    cwm = torch.tensor(cwm, dtype=torch.bool)

    model = model.to(device)
    theta = model(mask, shirt)

    warped = F.grid_sample(shirt, theta, padding_mode='border', align_corners=True).to('cpu')
    warped = warped / 2 + 0.5
    warped = warped * cwm

    cwm = ~cwm

    person = cloud_person * cwm + warped
    # wc = transforms.Resize((256 + 256 // 2, 192 + 192 // 2))(wc)
    # wc = transforms.CenterCrop((256, 192))(wc)
    #warped_mask = np.array(topil(warped_mask[0]))
    # warped = np.array(topil(warped[0]))
    #wc = np.array(topil(wc[0]))

    person = topil(person[0]).convert('RGB')

    person.save(path_people + "2\\" + cloud_people[index])

C:\Users\Даша\AppData\Local\Temp\ipykernel_4228\3884460301.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cwm = torch.tensor(cwm, dtype=torch.bool)


KeyboardInterrupt: 